In [2]:
import os
import pathlib
import urllib.request
import zipfile

def prepare_plugin(anti_captcha_api_key: str, proxy_settings: dict):
    current_script_dir = os.getcwd()
    modified_plugin_path = os.path.join(current_script_dir, 'plugin_hardcoded_api_key.zip')
    plugin_extract_path = os.path.join(current_script_dir, 'extracted_plugin')

    url = 'https://antcpt.com/anticaptcha-plugin.zip'
    filehandle, _ = urllib.request.urlretrieve(url)

    # Unpack it
    with zipfile.ZipFile(filehandle, "r") as f:
        f.extractall(path=plugin_extract_path)

    config_path = os.path.join(plugin_extract_path, 'js', 'config_ac_api_key.js')
    with open(config_path, 'r') as file:
        config_content = file.read()
    
    config_content = config_content.replace("antiCapthaPredefinedApiKey = ''", f"antiCapthaPredefinedApiKey = '{anti_captcha_api_key}'")
    
    # Add proxy settings
    proxy_config = f"""
    solveProxyOnTasks = true;
    userProxyProtocol = '{proxy_settings["protocol"]}';
    userProxyServer = '{proxy_settings["server"]}';
    userProxyPort = {proxy_settings["port"]};
    userProxyLogin = '{proxy_settings["login"]}';
    userProxyPassword = '{proxy_settings["password"]}';"""
    
    with open(config_path, 'w') as file:
        file.write(config_content + proxy_config)

    # Repack the plugin directory
    zip_file = zipfile.ZipFile(modified_plugin_path, 'w', zipfile.ZIP_DEFLATED)
    for root, dirs, files in os.walk(plugin_extract_path):
        for file in files:
            path = os.path.join(root, file)
            zip_file.write(path, arcname=path.replace(plugin_extract_path, ''))
    zip_file.close()

    return modified_plugin_path


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


def get_browser_with_plugin_ready(driver_path: str, api_key: str, proxy_settings: dict):
    options = webdriver.ChromeOptions()
    options.page_load_strategy = 'none'
    
    plugin_path = prepare_plugin(api_key, proxy_settings)
    options.add_extension(plugin_path)
    
    return webdriver.Chrome(service=Service(driver_path), options=options)


proxy_settings = {
    "protocol": "http",
    "server": "15.229.77.235",
    "port": 80,
    "login": "",
    "password": ""
}

navegador = get_browser_with_plugin_ready(
    driver_path='./chromedriver.exe',  
    api_key='daa5bc4640585af447aa4b2c7dc98175',
    proxy_settings=proxy_settings
)

navegador.get('https://chat.openai.com/')


In [6]:
navegador.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[2]/div[1]/div/div/button[1]').click()